<a href="https://colab.research.google.com/github/chisombrown/hair_classification/blob/main/hair_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

print(os.listdir())

Mounted at /content/gdrive
['.config', 'gdrive', 'sample_data']


In [2]:
import numpy as np
import matplotlib
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.colab import drive
drive.mount('/content/gdrive')
from keras.datasets import mnist
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel
from os import listdir
from os.path import isfile, join
from skimage.io import imread
from skimage.color import rgb2grey
from skimage.transform import resize
from sklearn.utils import shuffle
from skimage.feature import hog

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
from tensorflow.keras.applications.vgg16 import decode_predictions

In [7]:
#access main dir
patch_dir = '/content/gdrive/MyDrive/Patch1k'
# hair_dir = '/content/gdrive/MyDrive/Patch1k/Hair'
# nonhair_dir = '/content/gdrive/MyDrive/Patch1k/NonHair'

#access path for hair and nonhair
hair_non_hair = [join(patch_dir, 'Hair' ),join(patch_dir, 'NonHair')]
# hair_non_hair = [join(patch_dir, hair_dir ),join(patch_dir, nonhair_dir)]
#initialise list of paths
train_images_paths = []
test_images_paths = []
train_labels = []
test_labels = []
for i, dir in enumerate(hair_non_hair):
  #get paths to test and train path within each hair and nonhair dirs
  test_dir = join(dir, 'Testing')
  train_dir = join(dir, 'Training')
  # test_dir = '/content/gdrive/MyDrive/Patch1k/Hair/Testing'
  # train_dir = '/content/gdrive/MyDrive/Patch1k/Hair/Training'
  #loop style taken from stackoverflow https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
  train_images_paths += [join(train_dir, f) for f in listdir(train_dir) if isfile(join(train_dir, f))]
  test_images_paths += [join(test_dir, f) for f in listdir(test_dir) if isfile(join(test_dir, f))]
  #assign label to paths
  train_labels += [i]*len(listdir(train_dir))
  test_labels += [i]*len(listdir(test_dir))
  # print(train_images_paths[0])





In [ ]:
def paths_to_images(image_paths):
  """
  This function converts paths to images and returns a stack of images
  """
  #initialise stack of images with each image just being 227x227 zeros
  images = np.zeros((len(image_paths),227,227))
  #looping through list of paths and converting to list of images
  for i, path in enumerate(image_paths):
        image = imread(path)
        #if image in colour tranform into black and white
        if len(image.shape) == 3:
            image = rgb2grey(image)
        images[i,:,:] = image
  return images

In [ ]:
#images in own arrays from files
# stack of train images
train_images = paths_to_images(train_images_paths)
#stack of test images
test_images = paths_to_images(test_images_paths)

# (train_X, train_y), (test_X, test_y) = mnist.load_data()

#save image patches and labels for easier access
np.save('/content/gdrive/MyDrive/train_images_patch',train_images)
np.save('/content/gdrive/MyDrive/test_images_patch',test_images)
np.save('/content/gdrive/MyDrive/train_labels_patch', train_labels)
np.save('/content/gdrive/MyDrive/test_labels_patch', test_labels)



In [3]:
#load saved image patches and labels and shuffle so not memorised

train_images = np.load('/content/gdrive/MyDrive/train_images_patch.npy')
train_labels = np.load('/content/gdrive/MyDrive/train_labels_patch.npy')
train_images, train_labels = shuffle(train_images, train_labels)

test_images = np.load('/content/gdrive/MyDrive/test_images_patch.npy')
test_labels = np.load('/content/gdrive/MyDrive/test_labels_patch.npy')
test_images, test_labels = shuffle(test_images, test_labels)

In [60]:
#Get tiny feats
def get_tiny_feats(images):
  """
  This function downsamples images and stacks them
  """
  #initialise stack of images with each image just being 16x16 zeros
  tiny_images = np.zeros((len(images),16,16))
  #looping through list of paths and converting to list of images
  for i, image in enumerate(images):        
        resized_image = resize(image, (16,16), anti_aliasing=True)
        tiny_images[i,:,:] = resized_image
  return tiny_images


In [85]:
#Classify tiny feats
classifier = ensemble.RandomForestClassifier(n_estimators = 100)

tiny_train_feats = get_tiny_feats(train_images)
tiny_test_feats = get_tiny_feats(test_images)


tiny_train_feats = np.reshape(tiny_train_feats, (train_images.shape[0],-1))
tiny_test_feats = np.reshape(tiny_test_feats, (test_images.shape[0],-1))

classifier.fit(tiny_train_feats, train_labels)

classifier.score(tiny_test_feats, test_labels)


0.47619047619047616

In [90]:
#Get hog feats
def get_hog_feats(images):
  """
  This function creates hog descriptors of the patches
  """
  z=2
  y=4
  descriptors_stack = []
  for i, image in enumerate(images):
    hog_descriptor = hog(image,pixels_per_cell=(y,y), cells_per_block=(z,z), feature_vector=True )
    #this makes it 2d 
    hog_descriptor.reshape((-1,len(hog_descriptor)))
    descriptors_stack.append(hog_descriptor)
  return descriptors_stack



In [91]:
#Classify hog feats
classifier = ensemble.RandomForestClassifier(n_estimators = 100)

hog_train_feats = get_hog_feats(train_images)
hog_test_feats = get_hog_feats(test_images)

hog_train_feats = np.reshape(hog_train_feats, (train_images.shape[0],-1))
hog_test_feats = np.reshape(hog_test_feats, (test_images.shape[0],-1))

classifier.fit(hog_train_feats, train_labels)
classifier.score(hog_test_feats, test_labels)




0.45714285714285713

In [83]:
hair_prediction = classifier.predict(hog_test_feats)
print(hair_prediction)
print(len(hair_prediction))

[1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1
 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1
 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1
 1 1 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0
 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1
 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 0 1 0
 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0
 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 1
 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1
 0 1 0 1 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1
 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 1
 1 1 0 0 0 0 0 0 0 1 0 0 0]
420


In [ ]:
#Get vgg feats
 code form https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/
#code from https://towardsdatascience.com/extract-features-visualize-filters-and-feature-maps-in-vgg16-and-vgg19-cnn-models-d2da6333edd0

feats_stack = []
for i, path in enumerate(train_images_paths):

  model = VGG16(weights='imagenet')   # load the model
  model = Model(inputs=model.input, outputs=model.get_layer('block4_pool').output)

  img_pth = train_images_paths[0]
  img = load_img(img_pth, target_size=(224, 224))#load image in shape 224x224
  img = img_to_array(img) # convert the image to an array
  # expand dimensions so that it represents a single 'sample'
  img = expand_dims(img, axis=0)
  # prepare the image (e.g. scale pixel values for the vgg)
  img = preprocess_input(img)
  # get feature map for first hidden layer
  feats = model.predict(img)
  feats_stack.append(feats)
  model.summary()